In [ ]:
import os
import subprocess
from threading import Timer
from queue import Queue

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ROOT_DIR = "/workspace/" if not is_colab() else "/content/"
REPO_URL = "https://huggingface.co/spaces/cagliostrolab/animagine-xl-3.1"
REPO_DIR = os.path.join(ROOT_DIR, "animagine-xl")

NGROK_TOKEN = ""
NGROK_SUBDOMAIN = ""
PORT = 7860

# os.environ["HF_TOKEN"] = ""
os.environ["IS_COLAB"] = "1"
os.environ["MODEL"] = "https://huggingface.co/cagliostrolab/animagine-xl-3.1/blob/main/animagine-xl-3.1.safetensors"
os.environ["CACHE_EXAMPLES"] = "1"

def clone_repository(url, directory, branch=None):
    subprocess.run(["git", "clone", url, directory], check=True)
    if branch:
        subprocess.run(["git", "checkout", branch], cwd=directory, check=True)

def install_dependencies(directory):
    dependencies = ["accelerate==0.27.2", "diffusers==0.26.3", "gradio==4.20.0",
                    "invisible-watermark==0.2.0", "spaces==0.24.0", "omegaconf==2.3.0", "timm==0.9.10"]
    if is_colab():
        subprocess.run(["pip", "install"] + dependencies, check=True)
    else:
        requirements_path = os.path.join(directory, "requirements.txt")
        subprocess.run(["pip", "install", "-r", requirements_path], check=True)

def setup_ngrok_tunnel(port, queue, auth_token, subdomain):
    ngrok.set_auth_token(auth_token)
    url = ngrok.connect(port, bind_tls=True, subdomain=subdomain)
    queue.put(url)

def main():
    if not os.path.exists(REPO_DIR):
        print(f"Cloning repository to {REPO_DIR}")
        clone_repository(REPO_URL, REPO_DIR)

    print("Installing required Python libraries")
    install_dependencies(REPO_DIR)
    print("Done!")

    os.chdir(REPO_DIR)

    if NGROK_TOKEN:
        try:
            from pyngrok import conf, ngrok
        except ImportError:
            subprocess.run(["pip", "install", "-qqqq", "--upgrade", "setuptools"], check=True)
            subprocess.run(["pip", "install", "-qqqq", "-U", "pyngrok"], check=True)
            from pyngrok import conf, ngrok

        ngrok.kill()
        ngrok_output_queue = Queue()
        ngrok_thread = Timer(2, setup_ngrok_tunnel, args=(PORT, ngrok_output_queue, NGROK_TOKEN, NGROK_SUBDOMAIN))
        ngrok_thread.start()
        ngrok_thread.join()
        print(ngrok_output_queue.get())

    !python app.py

if __name__ == "__main__":
    main()